In [5]:
#from code given, used in preprocessing etc
import numpy as np #scientific computing
import pandas as pd #data manipulation and analysis
import sys #used for handling command-line arguments
import re # for preprocessing info column, regex


#database related things
import os #to read environment variables
import sqlalchemy #for interacting with databases
from sqlalchemy import create_engine,types
import tqdm # to show progress bar
import time # to compare insertion method efficiency
from dotenv import load_dotenv # to load environment variables from .env file
load_dotenv()# write details like host, user, password, dbname in .env file, this loads them into this script

True

In [6]:
all_dataframes=[]
csv_file_path = 'finalData'
for filename in os.listdir(csv_file_path):
    if filename.endswith('.csv'):
        # Read the CSV file into a dataframe
        file_path = os.path.join(csv_file_path, filename)
        df = pd.read_csv(file_path)
        # Skip headers for dataframes after the first one
        if len(all_dataframes) > 0:
            df = df.iloc[1:]
        # Append the dataframe to the list
        all_dataframes.append(df)

/var/folders/wt/j2yy846s7sq12fdd3pl1mlnh0000gn/T/ipykernel_26953/352998408.py:7: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [7]:
for i in range(15):
    print(all_dataframes[i].shape)

(469943, 22)
(441016, 22)
(699057, 22)
(634826, 22)
(627089, 22)
(710175, 22)
(653293, 22)
(526070, 22)
(597088, 22)
(613583, 22)
(381736, 22)
(687336, 22)
(825065, 22)
(984443, 22)
(625832, 22)


In [8]:
db_name = os.getenv("MYSQL_DB_NAME")
user = os.getenv("MYSQL_DB_USER")
password = os.getenv("MYSQL_DB_PASSWORD")
host = os.getenv("MYSQL_DB_HOST")
port = os.getenv("MYSQL_DB_PORT")
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{db_name}')
startTime = time.time()
for df in tqdm.tqdm(all_dataframes):
    # info column longer than normal, so increasing it's length to 65535
    #max length in current 15 files for info column is: 107983
    dtype = {'INFO': types.TEXT(length=65535)} # dont need to mention other ones for now
    df.to_sql('model1', con=engine, if_exists='append', index=False, dtype=dtype)
    # Commit changes and close the connection
engine.dispose()
endTime = time.time()
Unspecified_to_sqlTime = endTime - startTime
print("Time taken to insert using to_sql: ", Unspecified_to_sqlTime)

100%|██████████| 65/65 [27:03<00:00, 24.98s/it]

Time taken to insert using to_sql:  1623.8379321098328


#### trying another method which is supposed to be faster but one iteration takes much longer than 13min, not even one df was inserted
#### seems to create table but not inserting anything(select* returns empty set)

In [9]:

startTime = time.time()
for df in tqdm.tqdm(all_dataframes):
    # info column longer than normal, so increasing it's length to 65535
    dtype = {'INFO': types.TEXT(length=65535)} # dont need to mention other ones for now
    df.to_sql('model2', con=engine, if_exists='replace', index=False, dtype=dtype,method='multi')
    # Commit changes and close the connection
engine.dispose()
endTime = time.time()
Multi_to_sqlTime = endTime - startTime
print("Time taken to insert using to_sql: ", Multi_to_sqlTime)

  0%|          | 0/65 [07:00<?, ?it/s]


PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)